# §6  Data Loading, Storage, and File Formats

Input and output typically falls into a few main categories: 
- reading text files and other more efficient on-disk formats, 
- loading data from databases, and 
- interacting with network sources like web APIs.

## § 6.1 Reading and Writing Data in Text Format
* Reading Text Files in Pieces
* Writing Data to Text Format
* Working with Delimited Formats
* JSON Data
* XML and HTML: Web Scraping

## § 6.2 Binary Data Formats
* Using HDF5 Format
* Reading Microsoft Excel Files

## § 6.3 Interacting with Web APIs
## § 6.4 Interacting with Databases
## § 6.5 Conclusion

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

#precision:控制打印精度(小數位數), suppress 是否要打印顯示小數位
np.set_printoptions(precision=4, suppress=True) 

## § 6.1 Reading and Writing Data in Text Format

###  in linux : !cat examples/ex1.csv

###  in windows : !type examples\ex1.csv

In [2]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [3]:
!type examples\ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


remember that other than values, there are columns and keys.
So, how would they be assigned, or would they?

In [4]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
pd.read_csv('examples/ex1.csv')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
!type examples\ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [7]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
# 額外提供 columns
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
# In the event that you want to form a hierarchical index from multiple columns, 
# pass a list of column numbers or names:

!type examples\csv_mindex.csv

parsed = pd.read_csv('examples/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [11]:
# 主要是以「空格」隔開
# In some cases, a table might not have a fixed delimiter, 
# using whitespace or some other pattern to separate fields.
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [12]:
# 連續的空格數目不定
# you can pass a regular expression as a delimiter for read_table.
result = pd.read_csv('examples/ex3.txt', sep='\s+') # \s+ : one or more space
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


↑ Because there was one fewer column name than the number of data rows,
read_table infers that the first column should be the DataFrame’s index in this special
case.

In [13]:
result.index

Index(['aaa', 'bbb', 'ccc', 'ddd'], dtype='object')

In [14]:
result.loc["aaa"]

A   -0.264438
B   -1.026059
C   -0.619500
Name: aaa, dtype: float64

In [15]:
!type examples\ex4.csv
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [16]:
# Missing data is usually either not present (empty string) or marked by some sentinel value.

!type examples\ex5.csv
result = pd.read_csv('examples/ex5.csv')
result

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [17]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [18]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [19]:
# The na_values option can take either a list or set of strings to consider missing values:

result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [20]:
# Different NA sentinels can be specified for each column in a dict: 

sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### § 6.1.1 Reading Text Files in Pieces

In [21]:
pd.options.display.max_rows = 10  # pandas 最大顯示 10 列

In [22]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [23]:
pd.read_csv('examples/ex6.csv', nrows=5) # read only a part of it

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [24]:
# the machine does not have enough memory to read the entire CSV
# Assuming you do not need the entire dataset in memory all at one time
# one way to avoid the problem would be to process the CSV in chunks
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

In [25]:
# we can iterate over ex6.csv, aggregating the value counts in the 'key' column like so:

chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

# fill_value  : add 前要在series中用 NaN替換的值
# value_count : 回傳一個所有不同數值即其數量的 Series, 回傳結果將按降序排列
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys


In [26]:
tot

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

In [27]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

# fill_value  : add 前要在series中用 NaN替換的值
# value_count : 回傳一個所有不同數值即其數量的 Series, 回傳結果將按降序排列
tot = pd.Series([])
for i, piece in enumerate(chunker):
    print(i)
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


0
1
2
3
4
5
6
7
8
9


In [28]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### § 6.1.2 Writing Data to Text Format

In [29]:
!type examples\ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [30]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [31]:
# 自動加上 implicit index
# 欠項：在兩個逗點之間、在遏點後，換行前
data.to_csv('examples/out.csv')
!type examples\out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [32]:
# writing to sys.stdout so it prints the text result to the console

import sys
t = data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [33]:
# ↑ Missing values appear as empty strings in the output. 
# You might want to denote them by some other sentinel value:

data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [34]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [35]:
# You can also write only a subset of the columns, and in an order of your choosing:

data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


### Warning : Series.to_csv() 預設 header 是開啟的，若沒有關掉會報 warning 提醒須提供 header

In [36]:
# Series also has a to_csv method:

dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)

# ts.to_csv('examples/tseries.csv')
ts.to_csv('examples/tseries.csv', header = False)

!type examples\tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### § 6.1.3 Working with Delimited Formats

In [37]:
!type examples\ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [38]:
ppp = pd.read_csv('examples\ex7.csv')
ppp

,a,b,c
0,1,2,3
1,1,2,3


In [39]:
ppp.dtypes

a    int64
b    int64
c    int64
dtype: object

In [40]:
# 另外一個，字串沒有用 quote
!type examples\ex77.csv

a,b,c
1,2,3
1,2,3


In [41]:
p77 = pd.read_csv('examples\ex77.csv')
p77

,a,b,c
0,1,2,3
1,1,2,3


In [42]:
p77.dtypes

a    int64
b    int64
c    int64
dtype: object

In [43]:
# delimiter 只有一個字元，而且資料用 quote
import csv
f = open('examples/ex7.csv')

reader = csv.reader(f)

In [44]:
reader

In [45]:
# list(reader)

In [46]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [47]:
# From there, it’s up to you to do the wrangling necessary 
# to put the data in the form that you need it.

with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [48]:
header, values = lines[0], lines[1:]

In [49]:
# *values 前面的 * 表示項次數目未定
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

###  CSV files come in many different flavors. To define a new format with a different delimiter, string quoting convention, or line terminator, we define a simple subclass of csv.Dialect:

```python
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
```

```python
reader = csv.reader(f, dialect=my_dialect)
```

```python
reader = csv.reader(f, delimiter='|')
```

## To write delimited files manually, you can use csv.writer.

```python
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))
```

### § 6.1.4 JSON Data

JSON (short for JavaScript Object Notation) has become one of the standard formats
for sending data by HTTP request between web browsers and other applications

- The basic types are objects (dicts), arrays (lists), strings, numbers, booleans, and nulls. 
- All of the keys in an object must be strings.

In [50]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [51]:
obj

'\n{"name": "Wes",\n "places_lived": ["United States", "Spain", "Germany"],\n "pet": null,\n "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},\n              {"name": "Katie", "age": 38,\n               "pets": ["Sixes", "Stache", "Cisco"]}]\n}\n'

## JSON Viewer
https://codebeautify.org/jsonviewer

In [52]:
type(obj)

str

In [53]:
# To convert a JSON string to Python form, use json.loads

import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [54]:
type(result)

dict

In [55]:
# json.dumps, on the other hand, converts a Python object back to JSON:
asjson = json.dumps(result) # dic 轉回 json

In [56]:
type(asjson)

str

In [57]:
result.keys()

dict_keys(['name', 'places_lived', 'pet', 'siblings'])

In [58]:
result['siblings']

[{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
 {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]

In [59]:
whole_siblings = pd.DataFrame(result['siblings'])

In [60]:
whole_siblings

,name,age,pets
0,Scott,30,"[Zeus, Zuko]"
1,Katie,38,"[Sixes, Stache, Cisco]"


In [61]:
whole_siblings.dtypes

name    object
age      int64
pets    object
dtype: object

In [62]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


In [63]:
!type examples\example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [64]:
# 其實不必像上面的過程要先取出 dictionary，而是可以直接轉為 dataframe

# The pandas.read_json can automatically convert JSON datasets in specific arrangements
# into a Series or DataFrame

data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [65]:
type(data)

pandas.core.frame.DataFrame

In [66]:
# If you need to export data from pandas to JSON, 
# one way is to use the to_json methods on Series and DataFrame
data.to_json()

'{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}'

In [67]:
type(data.to_json())

str

In [68]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [69]:
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [70]:
print(data.to_json(orient='records')) # no index

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


In [71]:
print(data.to_json) # no index

<bound method NDFrame.to_json of    a  b  c
0  1  2  3
1  4  5  6
2  7  8  9>


### § 6.1.5 XML and HTML: Web Scraping

- pandas has a built-in function, read_html, which uses libraries like lxml and Beautiful Soup to automatically parse tables out of HTML files as DataFrame objects. 
- you must install some additional libraries used by read_html:
```shell
$conda install lxml
$pip install beautifulsoup4 html5lib
```

## <table>（表格元素）
HTML <table> 元件代表表格數據──換句話說，就是透過二維資料表來呈現資訊。
    
https://developer.mozilla.org/zh-TW/docs/Web/HTML/Element/table

## Failed Bank List
This list includes banks which have failed since October 1, 2000.
https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/index.html

```html
<div class="dataTables_wrapper" id="table_wrapper"><div class="top"><div id="table_filter" class="dataTables_filter">Search: <input type="text"></div><div id="table_length" class="dataTables_length">Show <select size="1" name="table_length"><option value="10">10</option><option value="25" selected="selected">25</option><option value="50">50</option><option value="100">100</option><option value="-1">All</option></select> entries</div></div><div class="clear"></div><div class="title"><div class="dataTables_info" id="table_info">Showing 1 to 547 of 547 entries</div><div class="dataTables_paginate paging_full_numbers" id="table_paginate"><span class="first paginate_button" id="table_first">First</span><span class="previous paginate_button" id="table_previous">Previous</span><span></span><span class="next paginate_button" id="table_next">Next</span><span class="last paginate_button" id="table_last">Last</span></div></div><div class="clear"></div><table id="table" class="tablesorter">
```

```html
</table><div class="fg-toolbar ui-widget-header ui-corner-bl ui-corner-br ui-helper-clearfix"><div class="dataTables_info">Showing 1 to 547 of 547 entries</div><div class="dataTables_paginate paging_full_numbers"><span class="first paginate_button">First</span><span class="previous paginate_button">Previous</span><span></span><span class="next paginate_button">Next</span><span class="last paginate_button">Last</span></div></div><div class="clear"></div></div>
```

In [72]:
# The pandas.read_html function has a number of options, but by default it searches
# for and attempts to parse all tabular data contained within <table> tags.

tables = pd.read_html('examples/fdic_failed_bank_list.html')
len(tables)

1

In [73]:
type(tables)

list

In [74]:
tables

[                             Bank Name             City  ST   CERT  \
 0                          Allied Bank         Mulberry  AR     91   
 1         The Woodbury Banking Company         Woodbury  GA  11297   
 2               First CornerStone Bank  King of Prussia  PA  35312   
 3                   Trust Company Bank          Memphis  TN   9956   
 4           North Milwaukee State Bank        Milwaukee  WI  20364   
 ..                                 ...              ...  ..    ...   
 542                 Superior Bank, FSB         Hinsdale  IL  32646   
 543                Malta National Bank            Malta  OH   6629   
 544    First Alliance Bank & Trust Co.       Manchester  NH  34264   
 545  National State Bank of Metropolis       Metropolis  IL   3815   
 546                   Bank of Honolulu         Honolulu  HI  21029   
 
                    Acquiring Institution        Closing Date  \
 0                           Today's Bank  September 23, 2016   
 1              

In [75]:
type(tables[0])

pandas.core.frame.DataFrame

In [76]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [77]:
len(tables)

1

In [78]:
tables[0].shape

(547, 7)

In [79]:
# computing the number of bank failures by year:

close_timestamps = pd.to_datetime(failures['Closing Date'])

In [80]:
close_timestamps.head()

0   2016-09-23
1   2016-08-19
2   2016-05-06
3   2016-04-29
4   2016-03-11
Name: Closing Date, dtype: datetime64[ns]

In [81]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

In [82]:
close_timestamps.dt.year

0      2016
1      2016
2      2016
3      2016
4      2016
       ... 
542    2001
543    2001
544    2001
545    2000
546    2000
Name: Closing Date, Length: 547, dtype: int64

In [83]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

In [84]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

## XML (eXtensible Markup Language) 
is another common structured data format supporting hierarchical, nested data with metadata.

## The lxml.etree Tutorial
https://lxml.de/tutorial.html

### Parsing XML with lxml.objectify

The New York Metropolitan Transportation Authority (MTA) publishes a number of data series about its bus and train services.

https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/mta_perf/Performance_MNR.xml

```XML
<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational
  systemwide. The availability rate is based on physical observations performed
  the morning of regular business days only. This is a new indicator the agency
  began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>

```

## XML Viewer
https://codebeautify.org/xmlviewer

In [85]:
# Using lxml.objectify, we parse the file and get a reference to the root node of the XML file 
# with getroot:

# https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/mta_perf/Performance_MNR.xml

from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()  # 取得根節點

In [86]:
type(root)

lxml.objectify.ObjectifiedElement

In [87]:
# root.INDICATOR returns a generator yielding each <INDICATOR> XML element
type(root.INDICATOR)

lxml.objectify.ObjectifiedElement

In [88]:
root == root.INDICATOR

False

In [89]:
root

<Element PERFORMANCE at 0x258da340148>

In [90]:
len(root)

1

In [91]:
root[0]

<Element PERFORMANCE at 0x258da340148>

In [92]:
root.INDICATOR

<Element INDICATOR at 0x258da340588>

In [93]:
len(root.INDICATOR)

648

In [94]:
for elt in root.INDICATOR:
    break

In [95]:
for child in elt.getchildren():
    print(child.tag) # sub-element
    print(child.pyval) # text content
#     break

INDICATOR_SEQ
28445
PARENT_SEQ

AGENCY_NAME
Metro-North Railroad
INDICATOR_NAME
On-Time Performance (West of Hudson)
DESCRIPTION
Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.

PERIOD_YEAR
2008
PERIOD_MONTH
1
CATEGORY
Service Indicators
FREQUENCY
M
DESIRED_CHANGE
U
INDICATOR_UNIT
%
DECIMAL_PLACES
1
YTD_TARGET
95.0
YTD_ACTUAL
96.9
MONTHLY_TARGET
95.0
MONTHLY_ACTUAL
96.9


In [96]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [97]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


In [98]:
perf.shape

(648, 12)

In [99]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot() # StringIO建立的是一個 file-like object，擁有 File Object的所有方法。

In [100]:
root
root.get('href')

'http://www.google.com'

In [101]:
root.text

'Google'

## § 6.2 Binary Data Formats

In [102]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [103]:
frame.to_pickle('examples/frame_pickle')

In [104]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### in linux : !rm examples/frame_pickle

### in windows : !del examples\frame_pickle

In [105]:
!del examples\frame_pickle

### § 6.2.1 Using HDF5 Format

In [106]:
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']

In [107]:
frame

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781
...,...
95,0.795253
96,0.118110
97,-0.748532
98,0.584970


In [108]:
store?

In [109]:
store.keys()

['/obj1', '/obj1_col']

In [110]:
store['obj1']

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781
...,...
95,0.795253
96,0.118110
97,-0.748532
98,0.584970


In [111]:
store['obj1_col']

0    -0.204708
1     0.478943
2    -0.519439
3    -0.555730
4     1.965781
        ...   
95    0.795253
96    0.118110
97   -0.748532
98    0.584970
99    0.152677
Name: a, Length: 100, dtype: float64

In [112]:
store.put('obj2', frame, format='table')  # 等同 store['obj2'] = frame

In [113]:
store['obj2']

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781
...,...
95,0.795253
96,0.118110
97,-0.748532
98,0.584970


In [114]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,1.007189
11,-1.296221
12,0.274992
13,0.228913
14,1.352917
15,0.886429


In [115]:
store.keys()

['/obj1', '/obj1_col', '/obj2']

In [116]:
store.close()

In [117]:
frame.to_hdf('mydata.h5', 'obj3', format='table')
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781


In [118]:
store = pd.HDFStore('mydata.h5') # open 'mydata.h5'

In [119]:
store.keys()

['/obj1', '/obj1_col', '/obj2', '/obj3']

In [120]:
store.close()

In [121]:
import os
os.remove('mydata.h5')

### § 6.2.2 Reading Microsoft Excel Files

In [122]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [123]:
xlsx

In [124]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [125]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [126]:
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [127]:
writer.close()

In [128]:
frame.to_excel('examples/ex2.xlsx')

In [129]:
!del examples\ex2.xlsx

## § 6.3 Interacting with Web APIs

In [130]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [131]:
type(resp)

requests.models.Response

In [132]:
data = resp.json()

In [133]:
# data

In [134]:
type(data)

list

In [135]:
len(data)

30

In [136]:
type(data[0])

dict

In [137]:
data[0]

{'url': 'https://api.github.com/repos/pandas-dev/pandas/issues/44349',
 'repository_url': 'https://api.github.com/repos/pandas-dev/pandas',
 'labels_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/44349/labels{/name}',
 'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/44349/comments',
 'events_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/44349/events',
 'html_url': 'https://github.com/pandas-dev/pandas/pull/44349',
 'id': 1046949444,
 'node_id': 'PR_kwDOAA0YD84uM4uK',
 'number': 44349,
 'title': 'TYP: misc typing in _libs',
 'user': {'login': 'twoertwein',
  'id': 6618166,
  'node_id': 'MDQ6VXNlcjY2MTgxNjY=',
  'avatar_url': 'https://avatars.githubusercontent.com/u/6618166?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/twoertwein',
  'html_url': 'https://github.com/twoertwein',
  'followers_url': 'https://api.github.com/users/twoertwein/followers',
  'following_url': 'https://api.github.com/users/twoertwein/following

In [138]:
data[0]['title']

'TYP: misc typing in _libs'

In [139]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,44349,TYP: misc typing in _libs,"[{'id': 1280988427, 'node_id': 'MDU6TGFiZWwxMj...",open
1,44348,CLN: address TODOs/FIXMEs,[],open
2,44347,ENH: Use find_stack_level universally,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
3,44346,REF: re-remove _putmask_preserve,[],open
4,44345,"BUG: frame.loc[2:, 'z'] not setting inplace wh...",[],open
...,...,...,...,...
25,44310,"BUG: groupby apply KeyError, hash instability?","[{'id': 233160, 'node_id': 'MDU6TGFiZWwyMzMxNj...",open
26,44306,PERF: DataFrame constructor from list dataclas...,"[{'id': 8935311, 'node_id': 'MDU6TGFiZWw4OTM1M...",open
27,44304,DOC: df.to_html documentation incorrectly cont...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
28,44303,ENH: GroupBy.get_group should accept lists wit...,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open


## § 6.4 Interacting with Databases

In [140]:
!del mydata.sqlite

In [149]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [150]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [151]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [152]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [153]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [154]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [155]:
con.close()
db.dispose()
!del mydata.sqlite

## § Conclusion